In [2]:
import pandas as pd
import numpy as np
import pycaret as pc
from utils import utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
#https://www.pyimagesearch.com/2021/05/06/implementing-feedforward-neural-networks-with-keras-and-tensorflow/




2021-09-19 13:57:28.530931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-19 13:57:28.530946: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# feature_df_9 = pd.read_csv('data/color_refined_molecules_9_hop.csv', index_col=0)
# feature_df_8 = pd.read_csv('data/color_refined_molecules_8_hop.csv', index_col=0)
feature_df_7 = pd.read_csv('data/color_refined_molecules_7_hop.csv', index_col=0)
feature_df_6 = pd.read_csv('data/color_refined_molecules_6_hop.csv', index_col=0)
feature_df_5 = pd.read_csv('data/color_refined_molecules_5_hop.csv', index_col=0)
feature_df_4 = pd.read_csv('data/color_refined_molecules_4_hop.csv', index_col=0)

# I'm just using all the colors and seeing what happens.
target_df = utils.load_graph_repersentations()[['target']]

In [4]:
df = pd.concat([feature_df_7,feature_df_6,feature_df_5,feature_df_4, target_df], axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# baseline model
import tensorflow as tf
from sklearn.metrics import precision_recall_curve, auc
from tensorflow.keras.optimizers import SGD


def compute_auprc(model, X_test, y_test):
    # https://stats.stackexchange.com/questions/157012/area-under-precision-recall-curve-auc-of-pr-curve-and-average-precision-ap
    precision, recall, thresholds = precision_recall_curve(y_test, model.predict(X_test))
    area = auc(recall, precision)
    return area

def get_optimizer():
	learning_rate = 0.1
	decay_rate = learning_rate / 300
	momentum = 0.8
	sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
	return sgd

# https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594
def create_baseline():
	# create model
	model = Sequential()
    # baseline is a single fully connected layer of the same size as the input deminsion
	model.add(Dense(1000, input_dim=20_000, activation='relu'))
	model.add(Dense(500, activation='relu'))
	model.add(Dense(64, activation='relu'))
	model.add(Dense(64, activation='relu'))
	model.add(Dense(64, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model

	#

	model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
	return model

In [20]:
# tf.config.list_physical_devices('GPU')
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
gpus

[]

In [14]:
from sklearn.model_selection import train_test_split

scores = []
X, Y = df.iloc[:,:-1].values, df.iloc[:,-1].values

for i in range(10):
    model = create_baseline()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=i)
    model.fit(X_train, y_train, validation_data=(X_test, y_test))
    score = compute_auprc(model, X_test, y_test)
    print(f'AUPRC : {score}')
    scores.append(score)



303/303 [==============================] - 27s 89ms/step - loss: 0.5235 - accuracy: 0.7356 - val_loss: 0.4785 - val_accuracy: 0.7706
AUPRC : 0.7368783564841948
303/303 [==============================] - 25s 82ms/step - loss: 0.5263 - accuracy: 0.7323 - val_loss: 0.4404 - val_accuracy: 0.7929
AUPRC : 0.7387766315626216
303/303 [==============================] - 26s 84ms/step - loss: 0.5369 - accuracy: 0.7282 - val_loss: 0.4672 - val_accuracy: 0.7528
AUPRC : 0.7381354905941304
303/303 [==============================] - 25s 83ms/step - loss: 0.5227 - accuracy: 0.7372 - val_loss: 0.4741 - val_accuracy: 0.7755
AUPRC : 0.719617090995448
303/303 [==============================] - 25s 83ms/step - loss: 0.5281 - accuracy: 0.7330 - val_loss: 0.4831 - val_accuracy: 0.7673
AUPRC : 0.7200050629938841
303/303 [==============================] - 26s 83ms/step - loss: 0.5218 - accuracy: 0.7304 - val_loss: 0.4548 - val_accuracy: 0.7826
AUPRC : 0.7243101363894098
289/303 [===========================>..] 

KeyboardInterrupt: 

AttributeError: module 'tensorflow._api.v2.train' has no attribute 'GradientDescentOptimizer'

# evaluate model